In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

wiki.name

'wikipedia'

In [9]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents[:20],OllamaEmbeddings(model="llama3.2"))
retriever=vectordb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7f1151ec3290>)

In [16]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever, "langsmith_search", "Search for information about LangSmith. Any Langsmith related question will be answered using this tool.")
retriever_tool.name

'langsmith_search'

In [13]:
from langchain_community.tools import ArxivQueryRun
from langchain_community.utilities import ArxivAPIWrapper

arxiv_wrapper= ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [17]:
tools=[wiki, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/home/dhirenrajesh/Programs/Langchain/lc-kn-venv/lib64/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. Any Langsmith related question will be answered using this tool.', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7f11719d4360>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectors

In [ ]:
from dotenv import load_dotenv
load_dotenv()
from langchain_community.chat_models import ChatOllama

llm=ChatOllama(model="llama3.2", temperature=0)

True

In [32]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
prompt

PromptTemplate(input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'], metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'react', 'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'}, template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}')

In [34]:
#Agents
from langchain.agents import create_react_agent, AgentExecutor
agent=create_react_agent(llm, tools, prompt=prompt)
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor.invoke({"input":"Tell me about Langsmith"})


Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Action: langsmith_search
Action Input: query='Langsmith'LangSmith docs - Docs by LangChainSkip to main contentDocs by LangChain home pageLangSmithSearch...⌘KAsk AIGitHubTry LangSmithTry LangSmithSearch...NavigationLangSmith docsGet startedObservabilityEvaluationPrompt engineeringDeploymentAgent BuilderPlatform setupReferenceOverviewPlansCreate an account and API keyAccount administrationOverviewSet up a workspaceManage organizations using the APIManage billingSet up resource tagsUser managementAdditional resourcesPolly (Beta)Data managementAccess control & AuthenticationScalability & resilienceFAQsRegions FAQPricing FAQLangSmith statusLangSmith docsCopy pageCopy pageLangSmith provides tools for developing, debugging, and deploying LLM applications.
It helps you trace requests, evaluate outputs, test prompts, and manage deployments in one place.
LangSmith is framework agnostic, so you can use it with or without LangChain’s open-source libraries
langchain and langgraph.

Copy the key and

{'input': 'Tell me about Langsmith',
 'output': "Langsmith is a framework for developing, debugging, and deploying Large Language Model (LLM) applications. It provides tools for tracing requests, evaluating outputs, testing prompts, and managing deployments in one place. Langsmith is framework agnostic, allowing users to use it with or without LangChain's open-source libraries, langchain and langgraph."}